[View in Colaboratory](https://colab.research.google.com/github/guitar79/OA-2018/blob/master/04_3_SDO_data_crawling_using_BeutifulSoup.ipynb)

**SDO data crawling (BeutifulSoup)**

* 이 자료는 [SDO](https://sdo.gsfc.nasa.gov/)의 HMII 자료를 ㅅ수ㅣㅂㄱ게 ㄷ다

* 이 자료는 아래 자료를 참고하여, python3 환경에서 돌아가도록 수정하여 작성하였다. 

* https://github.com/seungwonpark/SunSpotTracker

* python이 처음이라면 [Python Basic](https://colab.research.google.com/drive/1PCOoDIKypPVX9KTItMOht1cl96cPmeR_?authuser=1#scrollTo=3g6o04iLM0AF), [Python packages](https://colab.research.google.com/drive/1-1wx2VPEyNe11bmgpSpwdQgrJASCAqdH?authuser=1)를 먼저 학습하기를 권한다.

경기과학고등학교 관측천문학 강좌를 위해 만들었으며  <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">크리에이티브 커먼즈 저작자표시-비영리-동일조건변경허락 4.0 국제 라이선스</a>에 따라 이용할 수 있음.

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="크리에이티브 커먼즈 라이선스" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a>

#Web crawling

이 수업에서는 크롤링을 손쉽게 할 수 있도록 도움을 주는 package로 "Beautifulsoup", "Reqeusts" 를 사용하였다.
두 라이브러리가 하는 역할은 다음과 같다.

* Beautifulsoup : HTML태그 등 컨텐츠를 가져온 뒤 사용자가 파싱하기 쉽게 도와주는 라이브러리
* Requests : 데이터를 요청하는 라이브러리

리이브러리가 설치되었다면 import 해보자.


In [0]:
from datetime import datetime, timedelta
import os
import urllib.request
from urllib.request import urlopen
from bs4 import BeautifulSoup

# Target site

SDO 사이트에서는 자료를 검색하여 다운로드 받을 수 있는 툴(https://sdo.gsfc.nasa.gov/data/aiahmi/)을 제공하고 있다.

SDO 자료가 저장되어 있는 주소(https://sdo.gsfc.nasa.gov/assets/img/browse/)에 접속하면 YYYY/MM/DD 형태의 폴더 안에 자료가 정리되어 있는 것을 볼 수 있다. 이 곳에 저장된 자료의 규칙성을 찾아 웹 크롤링으로 쉽게 다운받을 수 있는 코드를 작성해 보자.

** Site structure anaysis **

먼저 [2012년 1월 1일](https://sdo.gsfc.nasa.gov/assets/img/browse/2012/01/01/)의 페이지를 열어보면 많은 파일들이 나열되어 있는데 파일 이름이 YYYYMMDD_HHmmSS_1024_xxxxx.jpg의 형태임을 알 수 있다. 각각의 의미를 다음과 같이 유추해 볼 수 있다.

* YYYYMMDD : date
* HHmmSS : time
* 1024 : image dimension
* xxxxx.jpg : image type.

파일명에서 날짜와 시간을 알 수 있으니 이를 이용하면 좋을 것으로 생각된다. 또한 파일명의 뒷부분은 내가 얻고자 하는 파일을 구별하는데 사용해 보자.

그러기 위해 몇몇 변수를 설정하자. 아래와 같이 변수를 설정하면 4 images/day 로 파일을 다운로드 받을 수 있을 것이다. 

In [2]:
# some variables for downloading (site, file, perid and time gap, etc.)
site = 'https://sdo.gsfc.nasa.gov/assets/img/browse/'
target = '1024_HMII.jpg' #this tpye of image will be downloading

startdate = '20180101' #start date
enddate = '20180102' #end date
time_gap = 6 #time gap
request_hour = range(0,24,time_gap) #make list
#request_hour = [0, 3, 6, 9, 12, 15, 18, 21] #make list

#variable for calculating date
start_date = datetime.date(datetime.strptime(startdate, '%Y%m%d')) #convert startdate to date type
end_date = datetime.date(datetime.strptime(enddate, '%Y%m%d')) #convert enddate to date type
duration = (end_date - start_date).days #total days for downloading
print ('*'*80)
print ((duration+1), 'days', int((duration+1)*(24/time_gap)), 'files will be downloaded')

********************************************************************************
2 days 8 files will be downloaded


python에서는 다음의 코드로 손쉽게 다운로드 받을 directory를 생성할 수 있다.

In [3]:
#make directory for saving files
save_folder = startdate + '-' + enddate + '-' + target[:-4] + '/'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)
    print ('*'*80)
    print (save_folder, 'is created')
else :
    print ('*'*80)
    print (save_folder, 'is exist')

********************************************************************************
20180101-20180102-1024_HMII/ is created


# Page analysis

이제 BeautifulSoup을 이용하여 본격적으로 web crawling을 해 보자. 이 사이트는 구조상 1일 단위로 폴더가 나누어져 있고, 그 안에 해당 날짜의 파일들을 나열해 놓은 형태를 띠고 있다.

다음의 코드는 url 변수에 지정한 날짜의 웹 페이지에 대해 html  source를 변수 soup 에 저장해 준다.

In [4]:
directory = '2018/01/01/'
#download_date = start_date
#directory = download_date.strftime('%Y') + '/' + download_date.strftime('%m') + '/' + download_date.strftime('%d') + '/'
url = site + directory
print ('trying %s ' %url)
print ('trying ... ', url)
# using BeutifulSoup for crowling
soup = BeautifulSoup(urlopen(url), "html.parser")
print('soup : ', soup)

trying https://sdo.gsfc.nasa.gov/assets/img/browse/2018/01/01/ 
trying ...  https://sdo.gsfc.nasa.gov/assets/img/browse/2018/01/01/
soup :  <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">

<html>
<head>
<title>Index of /assets/img/browse/2018/01/01</title>
</head>
<body>
<h1>Index of /assets/img/browse/2018/01/01</h1>
<pre>      <a href="?C=N;O=D">Name</a>                                 <a href="?C=M;O=A">Last modified</a>      <a href="?C=S;O=A">Size</a>  <a href="?C=D;O=A">Description</a><hr/>      <a href="/assets/img/browse/2018/01/">Parent Directory</a>                                          -   
      <a href="20180101_000000_1024_HMIB.jpg">20180101_000000_1024_HMIB.jpg</a>        2017-12-31 19:30  392K  
      <a href="20180101_000000_1024_HMIBC.jpg">20180101_000000_1024_HMIBC.jpg</a>       2017-12-31 19:30  492K  
      <a href="20180101_000000_1024_HMIIC.jpg">20180101_000000_1024_HMIIC.jpg</a>       2017-12-31 19:30  278K  
      <a href="20180101_000000_1024_HMIIF.

변수 soup 을 분석하여 파일 리스트를 얻기 위해서는 어떤 tag를 불러와야 하는지 찾아보자. 

파일 리스트는 < pre > 라고 써 있는 테그 안에 들어 있음을 확인할 수 있다. 다음은 < pre > 테그 안에 있는 것들만 list로 저장하는 코드이다. 

In [5]:
pre_list = soup.find_all('pre')
print('length of pre_list : ', len(pre_list))
print('pre_list', pre_list)

length of pre_list :  1
pre_list [<pre>      <a href="?C=N;O=D">Name</a>                                 <a href="?C=M;O=A">Last modified</a>      <a href="?C=S;O=A">Size</a>  <a href="?C=D;O=A">Description</a><hr/>      <a href="/assets/img/browse/2018/01/">Parent Directory</a>                                          -   
      <a href="20180101_000000_1024_HMIB.jpg">20180101_000000_1024_HMIB.jpg</a>        2017-12-31 19:30  392K  
      <a href="20180101_000000_1024_HMIBC.jpg">20180101_000000_1024_HMIBC.jpg</a>       2017-12-31 19:30  492K  
      <a href="20180101_000000_1024_HMIIC.jpg">20180101_000000_1024_HMIIC.jpg</a>       2017-12-31 19:30  278K  
      <a href="20180101_000000_1024_HMIIF.jpg">20180101_000000_1024_HMIIF.jpg</a>       2017-12-31 19:30  326K  
      <a href="20180101_000000_2048_HMIB.jpg">20180101_000000_2048_HMIB.jpg</a>        2017-12-31 19:30  1.6M  
      <a href="20180101_000000_2048_HMIBC.jpg">20180101_000000_2048_HMIBC.jpg</a>       2017-12-31 19:30  2.0M 

변수 pre_list 에서 파일 리스트를 얻기 위해서는 < a href = ...>  tag를 분리하여 file_list로 저장하는 코드이다.

In [6]:
file_list = pre_list[0].find_all('a')
print('length of file_list : ', len(file_list))
for i in range(10):
    print('file_list', i, file_list[i])

length of file_list :  9885
file_list 0 <a href="?C=N;O=D">Name</a>
file_list 1 <a href="?C=M;O=A">Last modified</a>
file_list 2 <a href="?C=S;O=A">Size</a>
file_list 3 <a href="?C=D;O=A">Description</a>
file_list 4 <a href="/assets/img/browse/2018/01/">Parent Directory</a>
file_list 5 <a href="20180101_000000_1024_HMIB.jpg">20180101_000000_1024_HMIB.jpg</a>
file_list 6 <a href="20180101_000000_1024_HMIBC.jpg">20180101_000000_1024_HMIBC.jpg</a>
file_list 7 <a href="20180101_000000_1024_HMIIC.jpg">20180101_000000_1024_HMIIC.jpg</a>
file_list 8 <a href="20180101_000000_1024_HMIIF.jpg">20180101_000000_1024_HMIIF.jpg</a>
file_list 9 <a href="20180101_000000_2048_HMIB.jpg">20180101_000000_2048_HMIB.jpg</a>


0 ~ 4번 자료는 파일 리스트가 아니므로 5번 자료부터 파일 이름으로 사용할 수 있을 것이다. 다음 코드는 tag를 제거하고 filename만 남겨준다.

In [7]:
for i in range(5, 15):
    filename = file_list[i].text
    print('filename : ', i, filename)

filename :  5 20180101_000000_1024_HMIB.jpg
filename :  6 20180101_000000_1024_HMIBC.jpg
filename :  7 20180101_000000_1024_HMIIC.jpg
filename :  8 20180101_000000_1024_HMIIF.jpg
filename :  9 20180101_000000_2048_HMIB.jpg
filename :  10 20180101_000000_2048_HMIBC.jpg
filename :  11 20180101_000000_2048_HMIIC.jpg
filename :  12 20180101_000000_2048_HMIIF.jpg
filename :  13 20180101_000000_256_HMIB.jpg
filename :  14 20180101_000000_256_HMIBC.jpg


# Downloading file

이제 filename을 얻었으면 아래 코드로 파일을 다운로드 받을 수 있을 것이다.

In [8]:
filename = '20180101_000000_1024_HMIB.jpg'
print ('url:', url+filename)
print ('save_folder:', save_folder)
urllib.request.urlretrieve(url+filename, '%s/%s' % (save_folder, filename))
print ('*'*60)
print ('Downloading ' + filename)

url: https://sdo.gsfc.nasa.gov/assets/img/browse/2018/01/01/20180101_000000_1024_HMIB.jpg
save_folder: 20180101-20180102-1024_HMII/
************************************************************


filename을 여러개 얻었다면 해당 날짜의 폴더에 있는 자료 목록에 대하여 loop (for or while)문으로 해당되는 파일만을 선택하여 다운로드 받을 수 있을 것이다. 

# using datetime

일 이름에 있는 날짜, 시각을 datetime 함수로 비교 조건으로 활용해 보자. 먼저 filename을 datetime으로 만들어 주는 함수를 정한다.

datetime은 strftime으로 string 형태로 변환할 수 있다. 아래 코드를 분석해 보자.

In [9]:
def filename_to_hour(filename):
    fileinfo = filename.split('_')
    return datetime.strptime(fileinfo[0]+fileinfo[1], '%Y%m%d%H%M%S')
filename = '20180101_000000_1024_HMIB.jpg'
print(type(filename_to_hour(filename)))
print(filename_to_hour(filename))
print(type(filename_to_hour(filename).strftime('%Y%m%d%H')), filename_to_hour(filename).strftime('%Y%m%d%H'))

<class 'datetime.datetime'>
2018-01-01 00:00:00
<class 'str'> 2018010100


이제 우리가 다운받고자 하는 날짜의 자료에 대해서 반복하여 실행하는 코드를 작성하여 완성해보자. 

스스로 완성해 보는 노력을 하기 바란다. 스스로 학습하는데 도움이 되도록 완성된 코드를 첨부한다.

* [완성된 코드](https://github.com/guitar79/OA-2018/blob/master/04-3.crawling_SDO_image_bf.py)